In [1]:
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def build_model(hp):
    units=hp.Int(name='units',min_value=16,max_value=64,step=16)
    model=keras.Sequential([
        layers.Dense(units,activation='relu'),
        layers.Dense(10,activation='softmax')
    ])
    optimizer=hp.Choice(name='optimizer',values=['adagrad','adam'])
    model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model

# Keras Tuner - HyperModel

In [5]:
import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int('units', min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation='relu'),
            layers.Dense(self.num_classes, activation='softmax')
        ])
        optimizer = hp.Choice('optimizer', values=['adagrad', 'adam'])
        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

hypermodel = SimpleMLP(num_classes=10)


In [7]:
tuner = kt.BayesianOptimization(hypermodel.build, objective='val_accuracy', max_trials=5)


In [8]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adagrad', 'conditions': [], 'values': ['adagrad', 'adam'], 'ordered': False}


# Model

In [9]:
# Load mnist dataset from keras
(xtrain, ytrain), (xtest, ytest) = keras.datasets.mnist.load_data()
xtrain = xtrain.reshape((-1, 28*28)).astype('float32') / 255
xtest = xtest.reshape((-1, 28*28)).astype('float32') / 255
xtrain_full = xtrain[:]
ytrain_full = ytrain[:]
num_val_samples = 10000
xtrain, xval = xtrain[:-num_val_samples], xtrain[-num_val_samples:]
ytrain, yval = ytrain[:-num_val_samples], ytrain[-num_val_samples:]
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]

tuner.search(
    x=xtrain, y=ytrain,
    batch_size=128,
    epochs=20,
    validation_data=(xval, yval),
    callbacks=callbacks,
    verbose=2
)


Trial 4 Complete [00h 00m 40s]
val_accuracy: 0.9736999869346619

Best val_accuracy So Far: 0.9736999869346619
Total elapsed time: 00h 02m 43s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
32                |64                |units
adam              |adam              |optimizer

Epoch 1/20
391/391 - 3s - loss: 0.5486 - accuracy: 0.8495 - val_loss: 0.2760 - val_accuracy: 0.9238 - 3s/epoch - 8ms/step
Epoch 2/20
391/391 - 2s - loss: 0.2664 - accuracy: 0.9252 - val_loss: 0.2197 - val_accuracy: 0.9387 - 2s/epoch - 5ms/step
Epoch 3/20
391/391 - 2s - loss: 0.2188 - accuracy: 0.9379 - val_loss: 0.1909 - val_accuracy: 0.9471 - 2s/epoch - 5ms/step
Epoch 4/20
391/391 - 2s - loss: 0.1887 - accuracy: 0.9458 - val_loss: 0.1756 - val_accuracy: 0.9514 - 2s/epoch - 5ms/step
Epoch 5/20
391/391 - 2s - loss: 0.1682 - accuracy: 0.9520 - val_loss: 0.1636 - val_accuracy: 0.9536 - 2s/epoch - 5ms/step
Epoch 6/20
391/391 - 2s - loss: 0.1528 - accuracy: 0.9558 - val_loss: 0.153

KeyboardInterrupt: 